# Compare accuracy of ICET with different distortion rectification methods  

In [13]:
from vedo import *
import os
from ipyvtklink.viewer import ViewInteractiveWidget
import pykitti
import numpy as np
import tensorflow as tf
import time
import pickle

#limit GPU memory ------------------------------------------------
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
  try:
    memlim = 4*1024
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=memlim)])
  except RuntimeError as e:
    print(e)
#-----------------------------------------------------------------

from tensorflow.math import sin, cos, tan
import tensorflow_probability as tfp
import sys
current = os.getcwd()
parent_directory = os.path.dirname(current)
sys.path.append(parent_directory)
from ICET_spherical import ICET
from utils import R_tf
from metpy.calc import lat_lon_grid_deltas
from pioneer.das.api.platform import Platform
from scipy.spatial.transform import Rotation as R
from pioneer.das.api.egomotion.imu_egomotion_provider import IMUEgomotionProvider as emp 
from matplotlib import pyplot as plt

%load_ext autoreload
%autoreload 2
%autosave 180
%matplotlib notebook

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosaving every 180 seconds


In [24]:
#Load test data
scan1 = np.load("sample_data/test1/rectified_linear_frame_2.npy")
scan2 = np.load("sample_data/test1/rectified_linear_frame_3.npy")

it = ICET(cloud1 = scan1, cloud2 = scan2, fid = 60, niter = 10, 
           draw = True, group = 2, RM = False, DNN_filter = False)
ViewInteractiveWidget(it.plt.window)


 estimated solution vector X: 
 tf.Tensor(
[ 1.5048624e+00  7.8456709e-03  3.9596175e-04 -1.3814781e-04
  6.5381982e-04 -2.3321042e-04], shape=(6,), dtype=float32)


ViewInteractiveWidget(height=1043, layout=Layout(height='auto', width='100%'), width=1280)

In [16]:
%%bash
dir

sample_data  test_gazebo_data.ipynb  vibration_comp.ipynb
